In [1]:
import gym, gym.spaces
from collections import namedtuple
import numpy as np 
from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn 
import torch.optim as optim 

In [2]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [3]:
e = gym.make("FrozenLake-v0")
e.observation_space

Discrete(16)

In [4]:
e.action_space

Discrete(4)

In [6]:
e.reset()

0

In [7]:
e.render()


SFFF
FHFH
FFFH
HFFG


In [17]:
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space,
                          gym.spaces.Discrete)
        shape = (env.observation_space.n, )
        self.observation_space = gym.spaces.Box(
            0.0, 1.0, shape, dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res

In [9]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )
    
    def forward(self, x):
        return self.net(x)

In [10]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [11]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [12]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))
    
    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))
    
    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [14]:
env = DiscreteOneHotWrapper(gym.make("FrozenLake-v0"))
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

In [15]:
net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-frozenlake-naive")

In [18]:
for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f"
          % (iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 0.8:
        print("Solved!")
        break

writer.close()

0: loss=0.000, reward_mean=0.0, reward_bound=0.0
1: loss=0.000, reward_mean=0.0, reward_bound=0.0
2: loss=0.000, reward_mean=0.0, reward_bound=0.0
3: loss=0.000, reward_mean=0.0, reward_bound=0.0
4: loss=0.000, reward_mean=0.0, reward_bound=0.0
5: loss=0.000, reward_mean=0.0, reward_bound=0.0
6: loss=0.000, reward_mean=0.0, reward_bound=0.0
7: loss=0.000, reward_mean=0.0, reward_bound=0.0
8: loss=0.000, reward_mean=0.0, reward_bound=0.0
9: loss=0.000, reward_mean=0.0, reward_bound=0.0
10: loss=0.059, reward_mean=0.0, reward_bound=0.0
11: loss=0.000, reward_mean=0.0, reward_bound=0.0
12: loss=0.000, reward_mean=0.0, reward_bound=0.0
13: loss=0.000, reward_mean=0.0, reward_bound=0.0
14: loss=0.000, reward_mean=0.0, reward_bound=0.0
15: loss=0.000, reward_mean=0.0, reward_bound=0.0
16: loss=0.000, reward_mean=0.0, reward_bound=0.0
17: loss=0.000, reward_mean=0.0, reward_bound=0.0
18: loss=0.000, reward_mean=0.0, reward_bound=0.0
19: loss=0.041, reward_mean=0.0, reward_bound=0.0
20: loss=0

KeyboardInterrupt: 